## Orcid API

[Orcid](https://members.orcid.org/api/resources/work-types) has three different types of APIs available for use. For the purposes of this tutorial, the Public API will be used. Both the Member API and the Public API for Orcid uses two-legged authorization. 

Two-legged authorization: A call is sent to Orcid API containing credential information and the API returns a token. The access token returned has a 20 year life span and can be used to perform searches and accessing other Orcid records. 

In [1]:
%%bash
curl -i -L -H "Accept: application/json" -d "client_id=APP-NPXKK6HFN6TJ4YYI" -d "client_secret=060c36f2-cce2-4f74-bde0-a17d8bb30a97" -d "scope=/read-public" -d "grant_type=client_credentials" "https://sandbox.orcid.org/oauth/token" >> toxen.txt >&2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   321    0   191  100   130    191    130  0:00:01 --:--:--  0:00:01  1088
HTTP/1.1 200 OK
Date: Thu, 08 Feb 2018 17:55:07 GMT
Content-Type: application/json;charset=UTF-8
Transfer-Encoding: chunked
Connection: keep-alive
Set-Cookie: __cfduid=df08738ef3b75491d2520b3c3a1593b401518112507; expires=Fri, 08-Feb-19 17:55:07 GMT; path=/; domain=.orcid.org; HttpOnly
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: 0
X-XSS-Protection: 1; mode=block
X-Frame-Options: DENY
X-Content-Type-Options: nosniff
Expect-CT: max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"
Server: cloudflare
CF-RAY: 3ea07542df713036-YYZ

{"access_token":"3fe88727-1010-4875-8dab-1ddfbca1dcb0","token_type":"bearer","refresh_token":"3554201b-8ae0-48bc-9a3f-f8ffd500d37a","expires_in":62850877

In a Unix based system, the curl request above can be easily performed. If you are using windows, [installing](https://superuser.com/questions/134685/run-curl-commands-from-windows-console) curl maybe useful for future projects. 

This should result in a file containing a JSON object holding access tokens. The result I received is the following:

### Searching by institution

When using the Public API, two source URLs ([Sandbox](https://pub.sandbox.orcid.org) vs. [Production](https://pub.orcid.org)) are available. This part of the tutorial will use the Production URL and perform a GET request which is expected to return an data in an XML format. 

The API is under constant development. As a result, the base url for API is followed by the version extension. 

In [6]:
import requests
head = {"access_token":"79bd2af3-8079-454e-9807-031545a9acac"}#match this to your returned access_token
ins = 'Harvard University'
url = 'https://pub.orcid.org/v2.1/search/?q=%s' % ins.replace(" ","+")
r = requests.get(url, headers=head)
r.text #shows the returned result in XML format

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<search:search num-found="839331" xmlns:search="http://www.orcid.org/ns/search" xmlns:common="http://www.orcid.org/ns/common">\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-6619-1685</common:uri>\n            <common:path>0000-0002-6619-1685</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0001-5837-3203</common:uri>\n            <common:path>0000-0001-5837-3203</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-1178-3085</common:uri>\n            <common:path>0000-0002-1178-3085</common:path>\n            <common:ho

The search method shown above can become quiet powerful as the url is concatenated with author names and other parameters such as Boolean values. The ability to also take a list of institutions, iterate through them, and generating a file containing Orcid data can also be useful. 

**You Try**
Modify the code in the above URL to search by Lamont **and** Harvard.  

In [7]:
#Below is a sample code - does your data match with this?
auth_name = 'Lamont'
url = 'https://pub.orcid.org/v2.1/search/?q=%s+AND+%s' % (auth_name, ins.replace(" ","+"))
r = requests.get(url, headers=head) 
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<search:search num-found="9" xmlns:search="http://www.orcid.org/ns/search" xmlns:common="http://www.orcid.org/ns/common">\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-0963-1779</common:uri>\n            <common:path>0000-0002-0963-1779</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0002-5089-9769</common:uri>\n            <common:path>0000-0002-5089-9769</common:path>\n            <common:host>orcid.org</common:host>\n        </common:orcid-identifier>\n    </search:result>\n    <search:result>\n        <common:orcid-identifier>\n            <common:uri>https://orcid.org/0000-0001-9400-3517</common:uri>\n            <common:path>0000-0001-9400-3517</common:path>\n            <common:host>or

The above record should show only one record for the given criteria. To match this record, you can always visit the [advanced search](https://orcid.org/orcid-search/search) feature on Orcid and validate. 

### Search by known ID

If you know the ORCID of a researcher and would like to fetch the necessary information a similar approach to above can be taken. This trick can be more powerful if there is a list of known IDs and you would like iterate through the list for each individual and collect data as a JSON object. 

If you want to work in JSON format instead of XML data format returned above can be done by serializing the data. Libraries such as json, xmltodict, ElementTree is handy for achieving this. 

In [8]:
BASE_URL = 'https://pub.orcid.org/v2.1/'
#must have a known, or set of known IDs
ORCID_ID = '0000-0002-8512-1308/'
# Pick an API Type (orcid-profile, orcid-bio, orcid-works, etc.)
url = BASE_URL + ORCID_ID
r = requests.get(url, headers=head)
r.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<record:record path="/0000-0002-8512-1308" xmlns:internal="http://www.orcid.org/ns/internal" xmlns:funding="http://www.orcid.org/ns/funding" xmlns:preferences="http://www.orcid.org/ns/preferences" xmlns:address="http://www.orcid.org/ns/address" xmlns:education="http://www.orcid.org/ns/education" xmlns:work="http://www.orcid.org/ns/work" xmlns:deprecated="http://www.orcid.org/ns/deprecated" xmlns:other-name="http://www.orcid.org/ns/other-name" xmlns:history="http://www.orcid.org/ns/history" xmlns:employment="http://www.orcid.org/ns/employment" xmlns:error="http://www.orcid.org/ns/error" xmlns:common="http://www.orcid.org/ns/common" xmlns:person="http://www.orcid.org/ns/person" xmlns:activities="http://www.orcid.org/ns/activities" xmlns:record="http://www.orcid.org/ns/record" xmlns:researcher-url="http://www.orcid.org/ns/researcher-url" xmlns:peer-review="http://www.orcid.org/ns/peer-review" xmlns:personal-details="http://www.orci

## Working with a Wrapper

An API wrapper is a library that increases a user's accessibility to an existing third-party API. The ORCID-Python wrapper library is an example of that. A Python wrapper is expected to return data in formats that are friendly for further manipulation. Please note that according to [official](https://pypi.python.org/pypi/orcid/) doc, this wrapper library has been perfected for the sandbox. Take caution about using this in future projects (specially in a production environment). 

In [9]:
import orcid
api = orcid.SearchAPI(sandbox=True)#change to False if you want Production API instead
search_results = api.search_public_generator('text:English', pagination=10)
first_result = next(search_results)
print(first_result)

ModuleNotFoundError: No module named 'orcid'

Orcid wrapper is better suited for memberAPI access because authenticating, creating new accounts, etc. can easily be achieved with the wrapper. Check out the orcid-python [github](https://github.com/ORCID/python-orcid) page for more information. 